In [45]:
import classylss.binding as CLASS
import numpy as np
import math

# dir(CLASS.ClassEngine())


In [46]:
def get_neutrino_masses(total_mass: float, hierarchy: str) -> np.ndarray:
    """Get the three neutrino masses, including the mass splittings.
        Hierarchy is 'inverted' (two heavy), 'normal' (two light) or degenerate."""
    #Neutrino mass splittings
    nu_M21 = 7.53e-5 #Particle data group 2016: +- 0.18e-5 eV2
    nu_M32n = 2.44e-3 #Particle data group: +- 0.06e-3 eV2
    nu_M32i = 2.51e-3 #Particle data group: +- 0.06e-3 eV2

    if hierarchy == 'normal':
        nu_M32 = nu_M32n
        #If the total mass is below that allowed by the hierarchy,
        #assign one active neutrino.
        if total_mass < np.sqrt(nu_M32n) + np.sqrt(nu_M21):
            return np.array([total_mass, 0, 0])
    elif hierarchy == 'inverted':
        nu_M32 = -nu_M32i
        if total_mass < 2*np.sqrt(nu_M32i) - np.sqrt(nu_M21):
            return np.array([total_mass/2., total_mass/2., 0])
    #Hierarchy == 0 is 3 degenerate neutrinos
    else:
        return np.ones(3)*total_mass/3.

    #DD is the summed masses of the two closest neutrinos
    DD1 = 4 * total_mass/3. - 2/3.*np.sqrt(total_mass**2 + 3*nu_M32 + 1.5*nu_M21)
    #Last term was neglected initially. This should be very well converged.
    DD = 4 * total_mass/3. - 2/3.*np.sqrt(total_mass**2 + 3*nu_M32 + 1.5*nu_M21+0.75*nu_M21**2/DD1**2)
    nu_masses = np.array([ total_mass - DD, 0.5*(DD + nu_M21/DD), 0.5*(DD - nu_M21/DD)])
    assert np.isfinite(DD)
    assert np.abs(DD1/DD -1) < 2e-2
    assert np.all(nu_masses >= 0)
    return nu_masses


In [47]:
m_nu = 0.1 # eV
hubble = 0.67
omega0 = 0.3
omegab = 0.05
ns = 0.96
scalar_amp = 2.1e-9
alpha_s = 0.0
w0_fld = -1.0
wa_fld = 0.0
N_ur = 0.
# Neff = 3.4

nu_hierarchy = 'normal'
nu_acc = 1e-5
redshift = 99
box = 100
npart = 75

In [48]:
pre_params = {
            'tol_background_integration': 1e-9, 'tol_perturb_integration' : 1.e-7,
            'tol_thermo_integration':1.e-5, 'k_per_decade_for_pk': 50,'k_bao_width': 8,
            'k_per_decade_for_bao':  200, 'neglect_CMB_sources_below_visibility' : 1.e-30,
            'transfer_neglect_late_source': 3000., 'l_max_g' : 50,
            'l_max_ur':150, 'extra metric transfer functions': 'y'}

        #Set the neutrino density and subtract it from omega0
omeganu = m_nu/93.14/hubble**2
omcdm   = (omega0 - omegab) - omeganu
gparams = {'h': hubble, 'Omega_cdm': omcdm,'Omega_b': omegab,
            'Omega_k': 0, 'n_s': ns, 'A_s': scalar_amp, 'alpha_s': alpha_s}

        #Lambda is computed self-consistently
if w0_fld != -1.0 or wa_fld != 0.:
    gparams['Omega_fld'] = 1 - omega0
    gparams['w0_fld'] = w0_fld 
    gparams['wa_fld'] = wa_fld
else:
    gparams['Omega_fld'] = 0
        # gparams['Omega_fld'] = 1 - omega0
        
        # gparams['w0_fld'] = w0_fld 
        # gparams['wa_fld'] = wa_fld
        # gparams['Omega_ur'] = omega_ur   # do not set Omega_ur if
        # set N_ur
numass = get_neutrino_masses(m_nu, nu_hierarchy)

        #Set up massive neutrinos
if m_nu > 0:
    print("cambfile: setting up massive neutrinos...")
    gparams['m_ncdm'] = '%.8f,%.8f,%.8f' % (numass[2], numass[1], numass[0])
    gparams['N_ncdm'] = 3
            # gparams['N_ur'] = 0.00641
            # gparams['N_ur'] = N_ur - 3
    # gparams['N_ur'] = N_ur
    gparams['N_eff'] = 3.9
            #Neutrino accuracy: Default pk_ref.pre has tol_ncdm_* = 1e-10,
            #which takes 45 minutes (!) on my laptop.
            #tol_ncdm_* = 1e-8 takes 20 minutes and is machine-accurate.
            #Default parameters are fast but off by 2%.
            #I chose 1e-5, which takes 6 minutes and is accurate to 1e-5
    gparams['tol_ncdm_newtonian'] = min(nu_acc,1e-5)
    gparams['tol_ncdm_synchronous'] = nu_acc
    gparams['tol_ncdm_bg'] = 1e-10
    gparams['l_max_ncdm'] = 50
            #This disables the fluid approximations, which make P_nu not match 
            # camb on small scales.
            #We need accurate P_nu to initialise our neutrino code.
    gparams['ncdm_fluid_approximation'] = 2
            #Does nothing unless ncdm_fluid_approximation = 2
            #Spend less time on neutrino power for smaller neutrino mass
    gparams['ncdm_fluid_trigger_tau_over_tau_k'] = 30000.* (m_nu / 0.4)
else:
            # gparams['N_ur'] = 3.046
    gparams['N_ur'] = N_ur # for mnu = 0, N_ur cannot be less than 3.046 in CLASS

        #Initial cosmology
pre_params.update(gparams)

maxk        = 2 * math.pi / box * npart * 8
powerparams = {'output': 'dTk vTk mPk', 'P_k_max_h/Mpc' : maxk, 
            "z_max_pk" : redshift + 1}
pre_params.update(powerparams)

        #At which redshifts should we produce CAMB output: we want the start and
        # end redshifts of the simulation, but we also want some other values
        # for checking purposes




        # 
        # pre_params['Omega_fld'] = 1 - omega0 + bg.Omega0_lambda  # so that Omega0_lambda == 0 (forced)


cambfile: setting up massive neutrinos...


In [49]:
engine  = CLASS.ClassEngine(pre_params)
# powspec = CLASS.Spectra(engine) # powerspec is an object

In [50]:
bg = CLASS.Background(engine)

In [51]:
dir(bg)

['C',
 'G',
 'H0',
 'N_ncdm',
 'N_ur',
 'Neff',
 'Omega0_b',
 'Omega0_cdm',
 'Omega0_dcdm',
 'Omega0_fld',
 'Omega0_g',
 'Omega0_k',
 'Omega0_lambda',
 'Omega0_m',
 'Omega0_ncdm',
 'Omega0_ncdm_tot',
 'Omega0_pncdm',
 'Omega0_pncdm_tot',
 'Omega0_r',
 'Omega0_ur',
 'Omega_b',
 'Omega_cdm',
 'Omega_fld',
 'Omega_g',
 'Omega_k',
 'Omega_lambda',
 'Omega_m',
 'Omega_ncdm',
 'Omega_pncdm',
 'Omega_r',
 'Omega_ur',
 'T0_cmb',
 'T0_ncdm',
 'T_cmb',
 'T_ncdm',
 '_RHO_',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'a_max',
 'a_today',
 'age0',
 'angular_diameter_distance',
 'comoving_distance',
 'comoving_transverse_distance',
 'compute_for_z',
 'data',
 'efunc',
 'efunc_prime',
 'h',
 'hubble_funct

In [52]:
bg.N_ur

3.9

In [53]:
# bg.N_eff

In [54]:
bg.Neff - bg.N_ur

3.039604882997471

In [55]:
3.046 - 3.0396048829974722

0.006395117002527595

In [56]:
bg.Omega0_g

5.508975898555038e-05

In [57]:
bg.Omega_g(0.)

5.5089758985550374e-05

In [58]:
bg.Omega0_r

0.00010481709225486054

In [59]:
bg.Omega0_r - bg.Omega0_g

4.972733326931016e-05

In [37]:
(bg.Omega0_r - bg.Omega0_g) / bg.Omega0_g / (7/8 * (4/11)**(4/3))

3.974597633663343

In [60]:
bg.Omega0_ur

4.879402083565384e-05

In [39]:
bg.Omega0_g + bg.Omega0_ur

9.517026277905835e-05

In [40]:
Omega_ur = 7/8 * (4/11)**(4/3) * 3.046 * bg.Omega0_g

In [41]:
7/8 * (4/11)**(4/3) * 3.046 * bg.Omega0_g

3.491285981853417e-05

In [42]:
Omega_ur

3.491285981853417e-05

In [44]:
5.3900210121286624e-05 - 7/8 * (4/11)**(4/3) * 3.046 * 5.29626e-05

1.7262331129323747e-05

In [9]:
import decimal

def round_redshift(z):
    if z < 1:
        rounded_z = round(z, 1)
    else:
        rounded_z = int(z)
    return rounded_z

# Example usage

print(round_redshift(0.123456789))


0.1


In [10]:
round_redshift(3.1)

3.1